In [1]:
import os
import time
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from keras import layers as L
from keras.models import Model
from keras.optimizers import Adam

2024-03-16 17:51:20.154585: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 17:51:20.154672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 17:51:20.156265: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 17:51:20.167781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 17:51:22.476263: W tensorflow/compiler/tf2

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if(physical_devices):
    tf.config.experimental.set_memory_growth(physical_devices[0] , True)
    print(f"GPU Connected : {physical_devices[0]}")
else:
    print("GPU Not Connected")

GPU Connected : PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-03-16 17:51:27.922601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-16 17:51:27.965261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-16 17:51:27.965448: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
IMG_H = 64
IMG_W = 64
IMG_C = 3

In [4]:
def create_dir(path):
    if not os.path.exists(path=path):
        os.makedirs(path)

In [5]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img)
    img = tf.image.resize(img, (IMG_H , IMG_W))
    img = tf.cast(img , tf.float32)
    img = (img - 127.5) / 127.5
    return img

In [6]:
def tf_dataset(images_path , batch_size):
    ds = tf.data.Dataset.from_tensor_slices(images_path)
    ds = ds.shuffle(buffer_size=1000).map(load_image , num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size=batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [7]:
def build_generator(latent_dim):
    noise = L.Input((latent_dim) , name="noise_input")
    
    x = L.Dense(256)(noise)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Dense(1024)(x)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Dense(4096)(x)
    x = L.LeakyReLU(0.2)(x)

    x = L.Dense(IMG_H * IMG_W * IMG_C)(x)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Reshape((IMG_H , IMG_W ,IMG_C))(x)
    
    fake_output = L.Activation("tanh")(x)
    
    return Model(noise , fake_output , name="generator")

In [8]:
def build_discriminator():
    inputs = L.Input((IMG_H , IMG_W , IMG_C) , name="disc_input")
    
    x = L.Flatten()(inputs)
    
    x = L.Dense(4096)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)
    
    x = L.Dense(1024)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)
    
    x = L.Dense(256)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)
    
    x = L.Dense(1)(x)
    
    return Model(inputs , x , name="discriminator")

In [9]:
from keras.losses import BinaryCrossentropy

@tf.function
def train_step(real_images , latent_dim , generator , discriminator , g_opt , d_opt):
    batch_size = tf.shape(real_images)[0]
    bce_loss = BinaryCrossentropy(from_logits=True , label_smoothing=0.1)
    
    # Discriminator
    noise = tf.random.normal([batch_size , latent_dim])
    for _ in range(2):
        with tf.GradientTape() as dtape:
            generated_images = generator(noise , training=True)
            
            real_output = discriminator(real_images , training=True)
            fake_output = discriminator(generated_images , training=True)
            
            d_real_loss = bce_loss(tf.ones_like(real_output) , real_output)
            d_fake_loss = bce_loss(tf.zeros_like(fake_output) , fake_output)
            d_loss = d_real_loss + d_fake_loss
            
            d_grad = dtape.gradient(d_loss , discriminator.trainable_variables)
            d_opt.apply_gradients(zip(d_grad , discriminator.trainable_variables))
            
    with tf.GradientTape() as gtape:
        generated_images = generator(noise , training=True)
        fake_output = discriminator(generated_images , training=True)
        
        g_loss = bce_loss(tf.ones_like(fake_output) , fake_output)
        
        g_grad = gtape.gradient(g_loss , generator.trainable_variables)
        g_opt.apply_gradients(zip(g_grad , generator.trainable_variables))
        
        
    return d_loss , g_loss            

In [10]:
def save_plot(examples , epoch , n):
    n = int(n)
    examples = (examples + 1) / 2.0
    examples = examples * 255
    file_name = f"samples/generated_plot_epoch-{epoch-1}.jpg"
    
    cat_image = None
    for i in range(n):
        start_idx = i * n
        end_idx = (i + 1) * n
        
        image_list = examples[start_idx:end_idx]
        if i == 0:
            cat_image = np.concatenate(image_list , axis = 1)
        else:
            tmp = np.concatenate(image_list , axis=1)
            cat_image = np.concatenate([cat_image , tmp] , axis=0)
            
    cv2.imwrite(file_name , cat_image) 

In [11]:
# Hyperparameter
batch_size = 8
latent_dim = 64
num_epochs = 50
n_samples = 100

In [12]:
images_path = glob("images/*.jpg")
images_path = images_path[0:1000]
print(f"Total Number of Images : {len(images_path)}")

Total Number of Images : 1000


In [13]:
create_dir("samples")
create_dir("saved_model")

In [14]:
# Model
g_model = build_generator(latent_dim=latent_dim)
d_model = build_discriminator()

2024-03-16 17:26:59.025721: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-16 17:26:59.025933: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-16 17:26:59.026052: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
print(g_model.summary())
print(d_model.summary())

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 noise_input (InputLayer)    [(None, 64)]              0         
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1024)              263168    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 4096)              4198400   
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 4096)              0 

In [16]:
from keras.optimizers import Adam

d_optimizer = Adam(learning_rate=0.0002 , beta_1=0.5)
g_optimizer = Adam(learning_rate=0.0002 , beta_1=0.5)

In [17]:
images_dataset = tf_dataset(images_path=images_path , batch_size=batch_size)
seed = np.random.normal(size=(n_samples , latent_dim))

In [18]:
for i in images_dataset:
    print(i)
    break

tf.Tensor(
[[[[ 0.27227473  0.2252159   0.30364728]
   [ 0.8053673   0.77067775  0.82266295]
   [ 0.54076093  0.49370214  0.5899031 ]
   ...
   [ 0.996571    0.98872787  0.999292  ]
   [ 0.27049392  0.2626508   0.31084797]
   [ 0.08356263  0.06358714  0.14178203]]

  [[ 0.81575906  0.76870024  0.8517545 ]
   [ 0.68674654  0.6396877   0.7309049 ]
   [ 0.11057368  0.06493087  0.1703776 ]
   ...
   [ 0.93722045  0.9293773   0.96772176]
   [ 0.77163565  0.7637925   0.80500585]
   [-0.14718759 -0.16716309 -0.08621946]]

  [[ 0.7438357   0.69677687  0.79297787]
   [ 0.12900199  0.08335918  0.18880591]
   [-0.17937395 -0.2225428  -0.11273888]
   ...
   [ 0.97988665  0.96996015  0.9961263 ]
   [ 0.9286296   0.91652447  0.9869284 ]
   [ 0.30889726  0.28084263  0.38389963]]

  ...

  [[ 0.9766228   0.9766228   0.9766228 ]
   [ 0.6064008   0.6064008   0.6064008 ]
   [ 0.7217132   0.7217132   0.7217132 ]
   ...
   [-0.5599121  -0.5834415  -0.42081898]
   [-0.17232403 -0.19585344 -0.04683383]
   [-

In [19]:
 for epoch in range(num_epochs):
        start = time.time()

        d_loss = 0.0
        g_loss = 0.0
        for image_batch in images_dataset:
            d_batch_loss, g_batch_loss = train_step(image_batch, latent_dim, g_model, d_model, g_optimizer, d_optimizer)
            d_loss += d_batch_loss
            g_loss += g_batch_loss

        d_loss = d_loss/len(images_dataset)
        g_loss = g_loss/len(images_dataset)

        g_model.save("saved_model/g_model.h5")
        d_model.save("saved_model/d_model.h5")

        examples = g_model.predict(seed, verbose=0)
        save_plot(examples, epoch, np.sqrt(n_samples))

        time_taken = time.time() - start
        print(f"[{epoch+1:1.0f}/{num_epochs}] {time_taken:2.2f}s - d_loss: {d_loss:1.4f} - g_loss: {g_loss:1.4f}")

2024-03-16 17:27:09.845152: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-16 17:27:10.872443: I external/local_xla/xla/service/service.cc:168] XLA service 0x777a4682cc50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-16 17:27:10.872523: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-03-16 17:27:10.884818: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-16 17:27:10.921813: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1710590231.066390  133504 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


/home/khabilan/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024-03-16 17:27:22.663731: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201326592 exceeds 10% of free system memory.


2024-03-16 17:27:23.100204: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201326592 exceeds 10% of free system memory.


[1/50] 16.51s - d_loss: 1.2777 - g_loss: 0.8358


2024-03-16 17:27:33.603252: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201326592 exceeds 10% of free system memory.


2024-03-16 17:27:34.105358: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201326592 exceeds 10% of free system memory.


[2/50] 13.45s - d_loss: 1.3982 - g_loss: 0.9222


2024-03-16 17:27:47.145384: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201326592 exceeds 10% of free system memory.


[3/50] 11.08s - d_loss: 1.0573 - g_loss: 1.0506
[4/50] 12.71s - d_loss: 1.2651 - g_loss: 1.1223
[5/50] 11.75s - d_loss: 1.1372 - g_loss: 1.2177
[6/50] 11.69s - d_loss: 1.2430 - g_loss: 1.3142
[7/50] 12.17s - d_loss: 1.2982 - g_loss: 1.2960
[8/50] 13.92s - d_loss: 1.1364 - g_loss: 1.3850
[9/50] 13.98s - d_loss: 1.1836 - g_loss: 1.4333
[10/50] 13.10s - d_loss: 1.1568 - g_loss: 1.5295
[11/50] 12.46s - d_loss: 1.2909 - g_loss: 1.4563
[12/50] 13.31s - d_loss: 1.1660 - g_loss: 1.4624
[13/50] 13.07s - d_loss: 1.0575 - g_loss: 1.4866
[14/50] 13.15s - d_loss: 1.1163 - g_loss: 1.5634
[15/50] 12.14s - d_loss: 1.3730 - g_loss: 1.5165
[16/50] 14.06s - d_loss: 1.2500 - g_loss: 1.3194
[17/50] 13.54s - d_loss: 1.1502 - g_loss: 1.4200
[18/50] 14.09s - d_loss: 1.1528 - g_loss: 1.5071
[19/50] 11.65s - d_loss: 1.1266 - g_loss: 1.4793
[20/50] 13.38s - d_loss: 1.0799 - g_loss: 1.3728
[21/50] 12.15s - d_loss: 1.1399 - g_loss: 1.4088
[22/50] 13.51s - d_loss: 1.1367 - g_loss: 1.2921
[23/50] 12.77s - d_loss: 1.